In [1]:
from src.woj_functions import *

# Demo

In [2]:
ask_woj('Jrue Holiday')

multiplier: 7 

Precise prediction from Woj: $15,137,050.25 

Woj_net says this year that Jrue Holiday should be paid between: $13,727,726.25 - $16,546,374.25 

Jrue Holiday actually paid $26,131,111 

Woj_net believes Jrue Holiday is getting overpaid this year by about $10,994,060.75


,Unnamed: 0,assist_percentage,assists,block_percentage,blocks,box_plus_minus,defensive_box_plus_minus,defensive_rebound_percentage,defensive_rebounds,defensive_win_shares,...,age,avg_salary,years_played,year_list,current_player,contract_total,contract_length,current_salary,current_avg_salary,current_team
468,468,31.8,518.0,1.8,54.0,3.0,-0.1,11.0,259.0,1.9,...,28.5,2.164676e+06,12,"['2009-10', '2010-11', '2011-12', '2012-13', '...",True,53151111.0,2.0,26131111.0,26575555.5,MIL
